In [ ]:
# Force CPU mode - MUST run this cell FIRST before any other cells
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = ''
print("✓ Forcing CPU mode (MPS disabled)")

In [1]:
import os 

In [4]:
%pwd

'/Users/sunilverma/Text-Summarizer-Project'

In [ ]:
#os.chdir("../")

In [6]:
%pwd

'/Users/sunilverma/Text-Summarizer-Project'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: str
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int

In [8]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [15]:
from pathlib import Path
from text_summarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from text_summarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
    ):
        # Convert relative paths to absolute
        if not config_filepath.is_absolute():
            config_filepath = Path(os.getcwd()) / config_filepath
        if not params_filepath.is_absolute():
            params_filepath = Path(os.getcwd()) / params_filepath
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        model_trainer_config = self.config.model_trainer
        training_args = self.params.TrainingArguments

        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(model_trainer_config.root_dir),
            data_path=Path(model_trainer_config.data_path),
            model_ckpt=Path(model_trainer_config.model_ckpt),
            num_train_epochs=training_args.num_train_epochs,
            warmup_steps=training_args.warmup_steps,
            per_device_train_batch_size=training_args.per_device_train_batch_size,
            weight_decay=training_args.weight_decay,
            logging_steps=training_args.logging_steps,
            evaluation_strategy=training_args.evaluation_strategy,
            eval_steps=training_args.eval_steps,
            save_steps=training_args.save_steps,
            gradient_accumulation_steps=training_args.gradient_accumulation_steps,
        )
        return model_trainer_config


In [16]:
from transformers import TrainingArguments, Trainer 
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk 
import torch

In [11]:
# from transformers import TrainingArguments, Trainer

# trainer_args = TrainingArguments(
#     output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
#     per_device_train_batch_size=1, per_device_eval_batch_size=1,
#     weight_decay=0.01, logging_steps=10,
#     eval_strategy='steps', eval_steps=500, save_steps=1e6,
#     gradient_accumulation_steps=16
# )

In [ ]:
class ModelTrainer: 
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        
        # Force CPU device explicitly
        device = "cpu"
        print(f"Using device: {device}")

        # Ensure Transformers logs show up and tqdm is enabled
        try:
            from transformers.utils import logging as hf_logging
            hf_logging.set_verbosity_info()
        except Exception:
            pass

        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt, use_safetensors=True)
        
        # Explicitly move model to CPU
        model = model.to(device)
        
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

        data_collator = seq2seq_data_collator

        # Loading the dataset from the data path
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        # Use 40 samples - calibrated for ~1 hour training on CPU
        train_dataset = dataset_samsum_pt['test'].select(range(min(40, len(dataset_samsum_pt['test']))))
        eval_dataset = dataset_samsum_pt['validation'].select(range(min(10, len(dataset_samsum_pt['validation']))))
        print(f"Training on {len(train_dataset)} samples")
        print(f"Evaluating on {len(eval_dataset)} samples")

        # Training configuration - optimized for ~1 hour completion
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=1,
            warmup_steps=10,
            per_device_train_batch_size=1,
            weight_decay=0.01,
            logging_strategy='steps',
            logging_steps=5,
            disable_tqdm=False,
            report_to='none',
            eval_strategy='steps',
            eval_steps=20,
            save_steps=int(1e6),
            gradient_accumulation_steps=4,
            no_cuda=True,
        )

        trainer = Trainer(
            model=model,
            args=trainer_args,
            tokenizer=tokenizer,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            data_collator=data_collator,
        )

        print("Starting training...")
        train_output = trainer.train()
        
        try:
            steps = int(train_output.global_step)
            print(f"Training finished. Total steps: {steps}")
        except Exception:
            pass
        print("Training completed!")

        # Saving the model after training
        model.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))



In [20]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e: 
    raise e

2025-12-19 10:38:32,754: INFO: common: yaml file: /Users/sunilverma/Text-Summarizer-Project/config/config.yaml loaded successfully
2025-12-19 10:38:32,765: INFO: common: yaml file: /Users/sunilverma/Text-Summarizer-Project/params.yaml loaded successfully
2025-12-19 10:38:32,767: INFO: common: created directory at: artifacts
Using device: cpu


loading configuration file config.json from cache at /Users/sunilverma/.cache/huggingface/hub/models--google--pegasus-cnn_dailymail/snapshots/40d588fdab0cc077b80d950b300bf66ad3c75b92/config.json
Model config PegasusConfig {
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 1,
  "forced_eos_token_id": 1,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {

Training on 3 samples for portfolio demo


Step,Training Loss
1,2.066900
2,3.166300
3,2.338200


Saving model checkpoint to artifacts/model_trainer/checkpoint-3
/Users/sunilverma/opt/anaconda3/envs/textS/lib/python3.10/site-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
Configuration saved in artifacts/model_trainer/checkpoint-3/config.json
Configuration saved in artifacts/model_trainer/checkpoint-3/generation_config.json
Model weights saved in artifacts/model_trainer/checkpoint-3/model.safetensors
tokenizer config file saved in artifacts/model_trainer/checkpoint-3/tokenizer_config.json
Special tokens file saved in artifacts/model_trainer/checkpoint-3/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration save

Training finished. Total steps: 3
Training completed!


Model weights saved in artifacts/model_trainer/pegasus-samsum-model/model.safetensors
tokenizer config file saved in artifacts/model_trainer/tokenizer/tokenizer_config.json
Special tokens file saved in artifacts/model_trainer/tokenizer/special_tokens_map.json
